In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from scipy.stats import truncnorm

In [ ]:
def generate_time_series(T, k, alpha, sigma, L, seed):
    """
    Generate $k$ time series of size $T$, each of them following an independent AR-1 process. 
    
    INPUT
    -----
    T (int) : length of time horizon
    k (int) : number of arms
    alpha (list): AR parameters 
    sigma (list): stochastic rates of change
    L (int): boundary
    seed (int) : random seed
    
    OUTPUT
    -----
    time_series : observed rewards of time series
    time_series_expected : expected rewards of time series at time $t$
    best_arm : arm with the highest expected reward at time $t$
    
    """

    np.random.seed(seed)
    
    time_series = np.zeros((k, T))
    time_series_expected = np.zeros((k, T))

    # Initial_values
    mu = [np.random.uniform(-L, L, k)]
    X = [mu[-1] + np.random.normal(0, sigma, k)]

    for t in range(1, T+1):
        mu.append([max(min(alpha[i] * X[-1][i], L),-L) for i in range(k)])
        
        # Update the actual rewards at t
        X_t = mu[-1] + np.random.normal(0, sigma, k)
        X.append(X_t)

    time_series = np.vstack(X).T
    time_series_expected = np.vstack(mu).T
    
    best_arm = np.argmax(time_series_expected, axis=0)
    
    return time_series, time_series_expected, best_arm 

In [ ]:
def compute_regret_percentage(time_series_expected, arms_selected, best_arm):
    """
    Compute the normalized regret given the arms selected
    """
    k, T = time_series_expected.shape
    assert (len(arms_selected) == T)
    assert (len(best_arm) == T)
    benchmark = np.array([time_series_expected[best_arm[t], t] for t in range(T)])
    true_rewards = np.array([time_series_expected[arms_selected[t], t] for t in range(T)])
    return (np.sum(benchmark) - np.sum(true_rewards))/np.sum(benchmark)

In [ ]:
def AR2(time_series, alpha, sigma, c_0, criteria=None, L = 5.0):
    """
    AR2 : Alteration-and-Restarting algorithm for dynamic AR bandits
    """
    alpha = np.array(alpha)
    
    arms_selected = []
    rewards = []
    UCB_lst = []
    
    V = set()
    c_1 = 8 * c_0

    k, T = time_series.shape

    ### Initialization ###
    for i in range(k):
        arms_selected.append(i)
        rewards.append(time_series[i, i])
        
    # gamma: our current belief for the expected reward of each arm at t = k
    gamma = np.array([alpha[i]**(k-i)*time_series[i, i] for i in range(k)])

    # tau: the last round at which that arm i was played
    # tau_trig: the last round at which arm i was last triggered
    tau = np.arange(k)
    tau_trig = np.zeros(k)
        
    for t in range(k, T):
        i_star = np.argmax(gamma) 
        gamma_star = gamma[i_star]
        
        confidence_bound_star = 0
        if alpha[i_star] == 1:
            confidence_bound_star = np.sqrt(t - tau[i_star] - 1)
        else:
            confidence_bound_star = np.sqrt((alpha[i_star]**2 - alpha[i_star]**(2*(t-tau[i_star])))/(1-alpha[i_star]**2))
    
        ### Step 1: Determine the triggering set ### 
        for j in set(range(k)).difference(V).difference({i_star}):
            if alpha[j] == 1:
                confidence_bound = np.sqrt(t - tau[j] - 1)
            else:
                confidence_bound = np.sqrt((alpha[j]**2 - alpha[j]**(2*(t-tau[j])))/(1-alpha[j]**2))
            
            # Check the triggering condition  
            if gamma_star - gamma[j] <= c_1 * sigma[j] * confidence_bound:
                # Add j to the activation set
                V.add(j)
                tau_trig[j] = t
                
        # compute the upper confidence bound for each arm
        UCB = [gamma[j] + c_1 * sigma[j] * np.sqrt((alpha[j]**2 - alpha[j]**(2*(t-tau[j])))/(1-alpha[j]**2)) \
                        if alpha[j] < 1 else gamma[j] + c_1 * sigma[j] * np.sqrt(t - tau[j] - 1) for j in range(k)]
        UCB_lst.append(UCB)
        UCB = np.array(UCB)

        ### Step 2: Exploit or Explore ###
        
        # Odd rounds: Play a triggered arm
        if t % 2 == 1 and len(V) > 0:
            V_list = list(V)
            if criteria == "UCB":
                # pick an activated arm with the highest UCB
                if np.max(UCB[V_list]) > UCB[i_star]:
                    i_t = np.array(V_list)[np.argmax(UCB[V_list])] 
                    V.remove(i_t)
                else:
                    i_t = i_star
            else:
                V_list = list(V)
                # pick a triggered arm with the earliest activation time
                i_t = np.array(V_list)[np.argmin(tau_trig[V_list])] 
                V.remove(i_t)
        # Even rounds: Play the superior arm
        else:
            i_t = i_star
            
        arms_selected.append(i_t)
        rewards.append(time_series[i_t, t])
        tau[i_t] = t
        
        # Update the new belief states
        gamma = alpha * gamma
        gamma[i_t] = max(min(alpha[i_t] * time_series[i_t, t], L), -L)
            
    return np.array(arms_selected), np.array(rewards), np.array(UCB_lst).T


In [ ]:
def ETC(time_series, m=50):
    """
    "Explore-then-commit" algorithm :  
    First select each arm for m rounds, then stick with the arm with the highest expected reward
    """
    print("running ETC")
    
    arms_selected = []
    rewards = []
    K, T = time_series.shape
    
    sum_rewards = np.zeros(K)
    
    for i in range(K):
        for j in range(m):
            current_t = i * m + j
            true_X = time_series[i, current_t]
            sum_rewards[i] += true_X
            arms_selected.append(i)
            rewards.append(true_X)
        
    i_star = np.argmax(sum_rewards)
    
    t = K * m
    
    while t < T:
        arms_selected.append(i_star)
        rewards.append(time_series[i_star, t])
        
        t += 1
        
    return np.array(arms_selected), np.array(rewards)

In [ ]:
def RExp3(time_series, V):
    """
    RExp3 from "Besbes et al. 2014"
    """ 
    K, T = time_series.shape 
    
    batch_size = int(np.ceil((K*np.log(K))**(1/3)*(T/V)**(2/3)))
    gamma = min(1, np.sqrt(K*np.log(K)/(np.exp(1)*batch_size)))
    
    print ("Batch size: {}".format(batch_size))
    
    arms_selected = []
    rewards = []
    
    j = 1
    while j <= np.ceil(T/batch_size):
        t_start = (j-1)*batch_size
        w = np.ones(K)
        for t in range(t_start, min(T, t_start + batch_size)):
            w_sum = np.sum(w)
            p = (1-gamma)*w/w_sum + (gamma/K)
            arm_t = np.random.choice(np.arange(K), p=p)
            reward_t = time_series[arm_t, t]
            w[arm_t] = w[arm_t]*np.exp(gamma*(reward_t/p[arm_t])/K)
            
            arms_selected.append(arm_t)
            rewards.append(reward_t)
            
        j += 1
        
    return np.array(arms_selected), np.array(rewards) 

In [ ]:
import random

def epsilon_greedy(time_series, alpha, epsilon=0.1):
    """
    Epsilon-greedy algorithm : act greedily with probability (1-epsilon), explore a random arm w.p. epsilon
    """
    print ("running eps-greedy")
    
    arms_selected = []
    rewards = []
    K, T = time_series.shape
    observed_idx = np.zeros(K)
    
    # Initialization
    for i in range(K):
        true_X = time_series[i, i]
        observed_idx[i] = i
        arms_selected.append(i)
        rewards.append(true_X)
        
    t = K
    while t < T:
        # w.p. (1-epsilon), act greedily
        if random.random() > epsilon:
            d = t - observed_idx
            expected_t = np.array([alpha[i]**d[i]*time_series[i, int(t-d[i])] for i in range(K)])

            i_t = np.argmax(expected_t)
        # w.p. epsilon, randomly explore any of the arms 
        else:
            i_t = random.randrange(K)
        
        observed_idx[i_t] = t
        arms_selected.append(i_t)
        rewards.append(time_series[i_t, t])
        
        t += 1
        
    return np.array(arms_selected), np.array(rewards)

In [ ]:
def UCB_simple(time_series):
    """
    UCB1 (Auer et al. 2002)
    """
    arms_selected = []
    rewards = []
    UCB = []

    ### Exploration -- Estimate the alpha's ###
    # K: number of arms
    # T: number of time steps
    K, T = time_series.shape
    mean = np.zeros(K)
    num_played = np.ones(K)

    # Initialization: play each arm once 
    for i in range(K):
        mean[i] = time_series[i, i]
        arms_selected.append(i)
        rewards.append(time_series[i,i])

    for t in range(K, T):
        UCB_t = mean + np.sqrt(2*np.log(t+1)/num_played)
        i_t = np.argmax(UCB_t)
        reward_t = time_series[i_t, t]

        mean[i_t] = (mean[i_t]*num_played[i_t] + reward_t)/(num_played[i_t] + 1)
        num_played[i_t] += 1

        arms_selected.append(i_t)
        rewards.append(time_series[i_t, t])
        UCB.append(list(UCB_t))
        
    return np.array(arms_selected), np.array(rewards), np.array(UCB).T

In [ ]:
def UCB_missing_data(time_series, alpha, sigma, delta=0.1):
    """
    modified UCB algorithm 
    """
    arms_selected = []
    rewards = []
    UCB = []

    ### Exploration -- Estimate the alpha's ###
    # K: number of arms
    # T: number of time steps
    K, T = time_series.shape
    observed_idx = np.zeros(K)

    ### Select each arm once as a starter ###
    for i in range(K):
        true_X = time_series[i, i]
        observed_idx[i] = i
        arms_selected.append(i)
        rewards.append(true_X)
        
    ### Exploitation -- use the UCB of X_t's to select the arm ###
    for t in range(K, T):
        d = t - observed_idx

        var = [(alpha[i]**2-alpha[i]**(2*(d[i])))/(1-alpha[i]**2) if alpha[i] < 1 else d[i]-1 for i in range(K)]
        bound = np.sqrt(2) * np.log(1/delta) * sigma * np.sqrt(var)
        UCB_t = np.array([(alpha[i])**d[i]*time_series[i, int(t-d[i])] + bound[i] for i in range(K)])

        i_t = np.argmax(UCB_t)
        
        observed_idx[i_t] = t

        arms_selected.append(i_t)
        rewards.append(time_series[i_t, t])
        UCB.append(list(UCB_t))
        
    return np.array(arms_selected), np.array(rewards), np.array(UCB).T

In [ ]:
def sliding_window_UCB(time_series, B, ksi=0.8):
    """
    sliding window UCB
    """
    K, T = time_series.shape
    
    arms_selected = np.zeros((K, T))
    
    rewards = []
    
    len_period = int(0.5 * np.sqrt(T))
    num_period = int(np.ceil(T/len_period))
    
    # length of the sliding window
    tau = int(2 * B * np.sqrt(T * np.log(T)/num_period))
    print ("length of the sliding window = {}".format(tau))
    
    for i in range(K):
        arms_selected[i, i] = 1
        rewards.append(time_series[i,i])
        
    for t in range(K, T):
        len_window = min(t, tau)
        history = arms_selected[:, t-len_window:t] # from time $t-tau$ to $t-1$
        N_t = np.sum(history, axis = 1)
        sum_X = np.sum(time_series[:, t-len_window:t] * history, axis = 1)
        mean_X = sum_X/N_t
        
        c_t = B * np.sqrt(ksi * np.log(len_window)) / np.sqrt(N_t)
        
        i_t = np.argmax(mean_X + c_t)
        arms_selected[i_t, t] = 1
        rewards.append(time_series[i_t, t])
    
    return np.nonzero(arms_selected.T)[1], np.array(rewards)

In [ ]:
def SW_TS(time_series, B):
    """
    sliding window Thompson sampling
    """
    K, T = time_series.shape
    
    ### For simplicity, we assume that the time series are bounded ###
    time_series = np.minimum(np.maximum(time_series, -L), L)
    
    arms_selected = np.zeros((K, T))
    
    rewards = []
    
    len_period = int(0.5 * np.sqrt(T))
    num_period = int(np.ceil(T/len_period))
    
    # length of the sliding window
    tau = int(2 * B * np.sqrt(T * np.log(T)/num_period))
    print ("length of the sliding window = {}".format(tau))
    
    for i in range(K):
        arms_selected[i, i] = 1
        rewards.append(time_series[i,i])
        
    for t in range(K, T):
        len_window = min(t, tau)
        history = arms_selected[:, t-len_window:t] # from time $t-tau$ to $t-1$
        
        # Number of times that arm i has been selected 
        N_t = np.sum(history, axis = 1)
        
        # Rewards collected by arm i within the window
        sum_X = np.sum(time_series[:, t-len_window:t] * history, axis = 1)/(2*L) + 1/2*N_t
        
        theta = [np.random.beta(sum_X[i]+1, N_t[i]-sum_X[i]+1) for i in range(i)]
           
        i_t = np.argmax(theta)
        arms_selected[i_t, t] = 1
        rewards.append(time_series[i_t, t])
        
    return np.nonzero(arms_selected.T)[1], np.array(rewards)
        

In [ ]:
### Compute regret for different-sized instances ###
T_max = 10000
num_simulations = 100

k = 10
L = 1

regret_AR2_1low = np.zeros(num_simulations)
regret_AR2_1high = np.zeros(num_simulations)

regret_ETC_low = np.zeros(num_simulations)
regret_ETC_high = np.zeros(num_simulations)

regret_RExp3_low = np.zeros(num_simulations)
regret_RExp3_high = np.zeros(num_simulations)
 
regret_eps_low = np.zeros(num_simulations)
regret_eps_high = np.zeros(num_simulations)

regret_UCB_low = np.zeros(num_simulations)
regret_UCB_high = np.zeros(num_simulations)

regret_UCB_mod_low = np.zeros(num_simulations)
regret_UCB_mod_high = np.zeros(num_simulations)

regret_SW_UCB_low = np.zeros(num_simulations)
regret_SW_UCB_high = np.zeros(num_simulations)

regret_SW_TS_low = np.zeros(num_simulations)
regret_SW_TS_high = np.zeros(num_simulations)


for n in range(num_simulations):
    print ("### simulation {} ###".format(n))
    
    np.random.seed(n + 100)
    
    # First generate alphas with expected values less than the threshold 
    alpha_1 = np.minimum(np.maximum(np.random.dirichlet(np.ones(k) * 5) + 0.4 - 1/k, 0), 1)
    print ("alpha-exp1:", alpha_1)
    sigma_1 = np.random.random(size=k) * 0.5
    print ("sigma-exp1:", sigma_1) 
    
    time_series_1, time_series_expected_1, best_arm_1 = \
        generate_time_series(T_max, k, alpha_1, sigma_1, L=L, seed=n)
    
    # Then generate alphas with expected values higher than the threshold 
    alpha_2 = np.minimum(np.maximum(np.random.dirichlet(np.ones(k) * 5) + 0.9 - 1/k, 0), 1)
    print ("alpha-exp2:", alpha_2)
    sigma_2 = np.random.random(size=k) * 0.5
    print ("sigma-exp2:", sigma_2) 
    
    time_series_2, time_series_expected_2, best_arm_2 = \
        generate_time_series(T_max, k, alpha_2, sigma_2, L=L, seed=n)
        
    arms_selected_AR2_1low, _, _ = AR2(time_series_1, alpha_1, sigma_1, 0.001, criteria="UCB", L = L)
    arms_selected_ETC_low, _ = ETC(time_series_1, m=100)
    arms_selected_RExp3_low, _ = RExp3(time_series_1, V=0.05*T_max)
    arms_selected_eps_low, _ = epsilon_greedy(time_series_1, alpha_1, epsilon=0.1)
    arms_selected_UCB_low, _, _ = UCB_simple(time_series_1)
    arms_selected_UCB_mod_low, _, _ = UCB_missing_data(time_series_1, alpha_1, sigma_1, delta=0.5)
    arms_selected_SW_UCB_low, _ = sliding_window_UCB(time_series_1, 2*L, ksi=0.8)
    arms_selected_SW_TS_low, _ = SW_TS(time_series_1, 2*L)
    
    regret_AR2_1low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                   arms_selected_AR2_1low[:T_max], \
                                                   best_arm_1[:T_max])
    
    regret_ETC_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                  arms_selected_ETC_low[:T_max], \
                                                  best_arm_1[:T_max])
    
    regret_RExp3_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                  arms_selected_RExp3_low[:T_max], \
                                                  best_arm_1[:T_max])
    
    regret_eps_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                  arms_selected_eps_low[:T_max], \
                                                  best_arm_1[:T_max])
    
    regret_UCB_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                  arms_selected_UCB_low[:T_max], \
                                                  best_arm_1[:T_max])
    
    regret_UCB_mod_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                      arms_selected_UCB_mod_low[:T_max], \
                                                      best_arm_1[:T_max])
    
    regret_SW_UCB_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                      arms_selected_SW_UCB_low[:T_max], \
                                                      best_arm_1[:T_max])
    
    regret_SW_TS_low[n] = compute_regret_percentage(time_series_expected_1[:, :T_max], \
                                                      arms_selected_SW_TS_low[:T_max], \
                                                      best_arm_1[:T_max])

    ####################################
        
    arms_selected_AR2_1high, _, _ = AR2(time_series_2, alpha_2, sigma_2, 0.1, criteria="UCB", L = L)
    arms_selected_ETC_high, _ = ETC(time_series_2, m=100)
    arms_selected_RExp3_high, _ = RExp3(time_series_2, V=0.05*T_max)
    arms_selected_eps_high, _ = epsilon_greedy(time_series_2, alpha_2, epsilon=0.1)
    arms_selected_UCB_high, _, _ = UCB_simple(time_series_2)
    arms_selected_UCB_mod_high, _, _ = UCB_missing_data(time_series_2, alpha_2, sigma_2, delta=0.5)
    arms_selected_SW_UCB_high, _ = sliding_window_UCB(time_series_2, 2*L, ksi=0.8)
    arms_selected_SW_TS_high, _ = SW_TS(time_series_2, 2*L)
    
    regret_AR2_1high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                   arms_selected_AR2_1high[:T_max], \
                                                   best_arm_2[:T_max])
    
    regret_ETC_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                  arms_selected_ETC_high[:T_max], \
                                                  best_arm_2[:T_max])
    
    regret_RExp3_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                  arms_selected_RExp3_high[:T_max], \
                                                  best_arm_2[:T_max])
    
    regret_UCB_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                  arms_selected_UCB_high[:T_max], \
                                                  best_arm_2[:T_max])
    
    regret_eps_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                  arms_selected_eps_high[:T_max], \
                                                  best_arm_2[:T_max])
    
    regret_UCB_mod_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                      arms_selected_UCB_mod_high[:T_max], \
                                                      best_arm_2[:T_max])
    
    regret_SW_UCB_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                      arms_selected_SW_UCB_high[:T_max], \
                                                      best_arm_2[:T_max])
    
    regret_SW_TS_high[n] = compute_regret_percentage(time_series_expected_2[:, :T_max], \
                                                      arms_selected_SW_TS_high[:T_max], \
                                                      best_arm_2[:T_max])
    

In [ ]:
print ("k = {}".format(k))
print ("Regret (low alpha)")
regret_low = \
[regret_AR2_1low, regret_ETC_low, regret_UCB_low, regret_eps_low, \
 regret_RExp3_low, regret_UCB_mod_low, regret_SW_UCB_low, regret_SW_TS_low]

for regret in regret_low:
    print ("{:0.2f} ({:0.2f})".format(np.mean(regret), np.std(regret)))    
    
print ("Regret (high alpha)")
regret_high = \
[regret_AR2_1high, regret_ETC_high, regret_UCB_high, regret_eps_high, \
 regret_RExp3_high, regret_UCB_mod_high, regret_SW_UCB_high, regret_SW_TS_high]

for regret in regret_high:
    print ("{:0.2f} ({:0.2f})".format(np.mean(regret), np.std(regret)))

In [ ]:
data = regret_high

fig, ax = plt.subplots()
bplot = ax.boxplot(data,
                  vert=True,  
                  patch_artist=True,  
                  )
ax.set_xticklabels(["AR2 \n", \
                    "ETC \n", \
                    "UCB \n", \
                    r"$\epsilon$"+"-greedy \n", \
                    "RExp3 \n", \
                    "mod-UCB \n", \
                    "SW-UCB \n", \
                    "SW-TS \n"
                   ])
ax.axvspan(0.5, 1.5, color='gray', alpha=0.2)
sns.set_style("whitegrid")

colors = ['pink'] + ['lightblue'] * (len(data)-1)
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

plt.ylabel('normalized regret', fontsize=16)

fig.set_size_inches(12,6)
plt.rcParams.update({'font.size': 15})
